In [1]:
from src.libraryAnalysier import LibraryAnalysier

import pandas as pd
import os
import numpy as np

In [2]:
data_dir = './data/100x20x3/'
OUTPUT_PATH = 'D:\\Musik\\Genres\\'
DIR_PATH = 'D:\\Musik\\Archive'
DIR_ERROR_PATH = DIR_PATH

In [3]:
from keras.models import load_model
reconstructed_model = load_model(os.path.join(data_dir, "model.h5"))

import joblib
loaded_scaler = joblib.load(os.path.join(data_dir, "scaler.pkl"))  # load from disk

import pickle
with open(os.path.join(data_dir, "labels.pkl"), "rb") as a_file:
    GENRES = pickle.load(a_file)

In [4]:
print(GENRES)

{0: 'Dancefloor', 1: 'Deep', 2: 'Jungle', 3: 'Liquid', 4: 'Minimal', 5: 'Neurofunk', 6: 'Sambass', 7: 'Shriek', 8: 'Techno'}


In [5]:
def loadGuess(q):
    a = 'other'
    a_val = 0
    b = 'other'
    b_val = 0
    
    temp = 0
    for i, r in enumerate(q):
        _max = np.max(r)
        temp += _max
        g = np.mean(r) * (1 - (np.var(r)))
        
        if((_max < 0.80)):
            continue
        if(g > a_val):
            b = a
            b_val = a_val
            a_val = g
            a = GENRES[i]
        elif(g > b_val):
            b = GENRES[i]
            b_val = g
    
    return a, a_val, b, b_val

In [6]:
def loadLabel(file, liblys):
    file_path = os.path.join(DIR_PATH, file)

    result = liblys.loadFile(file_path, None) 

    inside_df = pd.DataFrame(result)
    inside_df.drop(['length'], axis=1, inplace=True)
    inside_df = pd.DataFrame(loaded_scaler.transform(inside_df), columns=inside_df.columns)
    inside_df.reset_index(drop=True)
    q = reconstructed_model.predict(inside_df).T

    return loadGuess(q)

In [7]:
files = os.listdir(DIR_PATH)
liblys = LibraryAnalysier(library=DIR_PATH, segments = 100)

for file in files:
    if(os.path.splitext(file)[-1] not in ['.wav', '.mp3']):
        print('not supported')
        continue

    print(f'Loading {file}...')

    try:
        a, a_val, b, b_val = loadLabel(file, liblys)
    except Exception as e:
        print(f' > error: {e.args}')
        continue

    if a == 'other':
        print(' > no genre found')
    if b != 'other':    
        print(f' > ({a}: {round(np.max(a_val)*100, 2)}% or {b}: {round(np.max(b_val)*100, 2)}%)')
    else:
        print(f' > ({a}: {round(np.max(a_val)*100, 2)}%)')

    path_dir = os.path.join(OUTPUT_PATH, a)
    path_file = os.path.join(DIR_PATH, file)
    new_file = os.path.join(path_dir, os.path.basename(file)),
    # if not os.path.exists(OUTPUT_PATH):
    #     os.makedirs(OUTPUT_PATH)
    # if not os.path.exists(path_dir):
    #     os.makedirs(path_dir)
    
    # print(path_file, new_file)
    # os.replace(path_file, new_file)

Initialized library: 
Loading 'Shak Out' Noisia Radio Cut [Cause 4 Concern]_378692870 - AKOV.mp3...
 > (Dancefloor: 46.07% or Jungle: 33.5%)
Loading 02. David Guetta - When Love Takes Over (Blame Remix)_33580893 - Benedict Gwala.mp3...
 > (Dancefloor: 69.59%)
Loading 04 Conrad Subs - Sweet Sweet.wav...
 > (Sambass: 100.0%)
Loading 123 ._Invite's Choice Podcast 328 - Denise Rabe (1).mp3...
 > error: ()
Loading 123 ._Invite's Choice Podcast 328 - Denise Rabe (2).mp3...
 > error: ()
Loading 123 ._Invite's Choice Podcast 328 - Denise Rabe (3).mp3...
 > error: ()
Loading 123 ._Invite's Choice Podcast 328 - Denise Rabe (4).mp3...
 > error: ()
Loading 123 ._Invite's Choice Podcast 328 - Denise Rabe (5).mp3...
 > error: ()
Loading 123 ._Invite's Choice Podcast 328 - Denise Rabe (6).mp3...
 > error: ()
Loading 123 ._Invite's Choice Podcast 328 - Denise Rabe (7).mp3...
 > error: ()
Loading 123 ._Invite's Choice Podcast 328 - Denise Rabe (8).mp3...
 > error: ()
Loading 12A Froidy - The Unknown Co

Traceback (most recent call last):
  File "D:\Devtools\miniconda\lib\site-packages\IPython\core\interactiveshell.py", line 3369, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\robin\AppData\Local\Temp\ipykernel_20568\3036157579.py", line 12, in <cell line: 4>
    a, a_val, b, b_val = loadLabel(file, liblys)
  File "C:\Users\robin\AppData\Local\Temp\ipykernel_20568\2212202117.py", line 4, in loadLabel
    result = liblys.loadFile(file_path, None)
  File "c:\Users\robin\.mega\Coding\Python\Genre-Classification\src\libraryAnalysier.py", line 99, in loadFile
    return alyis.calculate()
  File "c:\Users\robin\.mega\Coding\Python\Genre-Classification\src\analysis.py", line 31, in calculate
    self.addFeatures()
  File "c:\Users\robin\.mega\Coding\Python\Genre-Classification\src\analysis.py", line 51, in addFeatures
    segment.calculate('tempo', segment.tempo)
  File "c:\Users\robin\.mega\Coding\Python\Genre-Classification\src\segment.py", line 17, in ca